In [1]:
import datetime
import numpy as np
import math
import os
import pandas as pd
import re
import seaborn as sns
import json
import statsmodels.api as sm 
import statsmodels.formula.api as smf
import statsmodels.graphics.api as smg
import sys

if sys.version_info[0] < 3: 
    from StringIO import StringIO
else:
    from io import StringIO
    
from IPython.display import HTML
def View(df):
    css = """<style>
    table { border-collapse: collapse; border: 3px solid #eee; }
    table tr th:first-child { background-color: #eeeeee; color: #333; font-weight: bold }
    table thead th { background-color: #eee; color: #000; }
    tr, th, td { border: 1px solid #ccc; border-width: 1px 0 0 1px; border-collapse: collapse;
    padding: 3px; font-family: monospace; font-size: 10px }</style>
    """
    s  = '<script type="text/Javascript">'
    s += 'var win = window.open("", "Title", "toolbar=no, location=no, directories=no, status=no, menubar=no, scrollbars=yes, resizable=yes, width=780, height=200, top="+(screen.height-400)+", left="+(screen.width-840));'
    s += 'win.document.body.innerHTML = \'' + (df.to_html() + css).replace("\n",'\\') + '\';'
    s += '</script>'
    return(HTML(s+css))    

os.chdir(r'C:\Users\User\GitHub\WebET_Analysis')
print("Current Working directory " , os.getcwd())

Current Working directory  C:\Users\User\GitHub\WebET_Analysis


In [2]:
# from IPython.display import HTML

# HTML('''<script>
# code_show=true; 
# function code_toggle() {
#  if (code_show){
#  $('div.input').hide();
#  } else {
#  $('div.input').show();
#  }
#  code_show = !code_show
# } 
# $( document ).ready(code_toggle);
# </script>
# <form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')

# Raw Data

## Search for specific subjects

In [3]:
#path = 'data_cognition'
#subject_files = os.listdir(path)
#for i in range(0, len(subject_files)):
#    thisSubject = open(path + "/" + subject_files[i]).read()
#    if thisSubject.find('5c0e4e3d87e876000151cfec') > (-1):
#        print(subject_files[i])

## Read CSV from String

In [4]:
def cleanhtml(raw_html):
    # https://stackoverflow.com/questions/9662346/python-code-to-remove-html-tags-from-a-string
    cleanr = re.compile('<.*?>')
    cleanText = re.sub(cleanr, '', raw_html)
    return cleanText

def cleanETText(text):
    textWithinBrackets = re.findall(re.compile('\[.*?\]'), text)
    output = text
    for i in range(0,len(textWithinBrackets)):
        old = textWithinBrackets[i]
        new = re.sub(",", "$", old)
        output = output.replace(old, new)
    return output

def cleanSurveyText(text):
    output = text
    textWithinBrackets = re.findall(re.compile('\{.*?\}'), text)
    for i in range(0,len(textWithinBrackets)):
        old = textWithinBrackets[i]
        new = old.replace(',', '§')
        output = output.replace(old, new)
    return output

def compileData(path):
    subject_files = os.listdir(path)
    all_subjects = []
    for i in range(0, len(subject_files)):
        csv_thisSubject = open(path + "/" + subject_files[i]).read()
        csv_thisSubject = cleanhtml(csv_thisSubject)
        csv_thisSubject = cleanETText(csv_thisSubject)
        csv_thisSubject = cleanSurveyText(csv_thisSubject)
        all_subjects.append(pd.read_csv(StringIO(csv_thisSubject)))
    output = pd.concat(all_subjects).reset_index(drop=True)
    return output

data_raw = compileData("data_prolific")

# Custom modifications
data_raw['run_id'].unique()

KeyboardInterrupt: 

In [ ]:
data_yang = compileData("data_yang2020WG")
# Custom modifications
data_yang.loc[0:515, 'run_id'] = 0
data_yang = data_yang.loc[~data_yang['run_id'].isin([4, 18]), :]
data_yang = data_yang.rename(columns={'eyeData': 'et_data'})
data_yang['run_id'] = data_yang['run_id'] + 1000
print(data_yang['run_id'].unique())
data_raw = data_raw.append(data_yang.reset_index(drop=True))

In [ ]:
data_milieu = compileData("data_milieu")
data_milieu['run_id'] = data_milieu['run_id'] + 2000
print(data_milieu['run_id'].unique())
data_raw = data_raw.append(data_milieu.reset_index(drop=True))

### Exclude empty studies

In [ ]:
data_raw = data_raw.loc[data_raw['trial_index']>0, :]

### survey data

In [ ]:
def cleanOptionalNote(text):
    optionalNoteText = re.findall(re.compile('optionalNote":.*?\}'), text)
    if len(optionalNoteText) < 1:
        output = text
    else:
        old = optionalNoteText[0]
        new = old.replace('§', ' ')
        output = text.replace(old, new)
    return output


def surveyStringToFrame(subject, string):
    string = cleanOptionalNote(string)
    string = re.sub("""{""", '', string)
    string = re.sub("""}""", '', string)
    string = re.sub('"', '', string)
    string = re.sub('§', '$', string)
    output = pd.read_csv(StringIO(string),
                         sep=":",
                         lineterminator="$",
                         header=None,
                         index_col=0
                         ).transpose()
    return output


def surveyData_thisSubject(data):
    df_thisSubject = data.loc[
        (pd.notna(data["responses"])) &
        (data["responses"] != '"'), :].reset_index()
    subject = df_thisSubject['run_id'].unique()[0]
    output = []
    for i in range(0, len(df_thisSubject)):
        output.append(
            surveyStringToFrame(subject,
                                df_thisSubject.loc[i, 'responses'])
        )

    output = pd.concat(output, axis=1)
    output['run_id'] = subject
    return output

def addSurveyData(data):
    surveyData_allSubjects = pd.DataFrame(columns=[
            'prolificID', 'age', 'gender', 'ethnic', 'sight', 
            'glasses', 'degree', 'eyeshadow', 'masquara', 'eyeliner', 
            'browliner', 'vertPosition', 'triedChin', 'keptHead', 
            'optionalNote', 'run_id'])

    for subject in data['run_id'].unique():
        surveyData_allSubjects = \
            surveyData_allSubjects.append(
                surveyData_thisSubject(
                    data.loc[data['run_id']==subject, ['run_id', 'responses']]
                )
            )
    # print(surveyData_allSubjects)
    
    data = data.merge(surveyData_allSubjects, on='run_id', how='left')
    data = data.drop(columns='responses')
    return(data)
    
data_raw = addSurveyData(data_raw)
print(data_raw['run_id'].unique())
print(data_raw.columns)

In [ ]:
def convertToNumeric(data, columns):
    data[columns] = data[columns].apply(pd.to_numeric, errors='coerce')
    return data


data_raw = convertToNumeric(data_raw, ['age'])
data_raw['degree'] = data_raw['degree'].replace(
        [
            'College / Undergraduate / Bachelor',
            'High School',
            'Graduate / PhD / Master',
            'Middle School'
        ],
    [
            'college',
            'highSchool',
            'grad',
            'middle'
        ]
)

columns = [
            'chinFirst',
            'eyeshadow', 
            'masquara',
            'eyeliner',
            'browliner',
            'triedChin', 
            'keptHead',
        ]

data_raw[columns] = data_raw[columns].replace({'no': 0, 'yes': 1}) 

data_raw['run_id'].unique()

# data_trial

In [ ]:
data_trial = data_raw.loc[
    :,
    [
        'run_id', 'subject', 'prolificID', 'chinFirst', 
        'trial_index',
        'trial_type', 'task_nr', 
        'rt', 'stimulus', 'key_press', 
        'time_elapsed', 'trial_duration', 'recorded_at',
        'window_width', 'window_height', 'success', 

        'chin', 'x_pos', 'y_pos', 
        
        'choiceTask_amountLeftFirst', 
        'option_topLeft', 'option_bottomLeft',
        'option_topRight', 'option_bottomRight', 
        'chosenAmount', 'chosenDelay',
    ]
]

## Clean

### Failed Webgazer Setup

In [ ]:
subjects_failedSetups = []
for subject in data_trial['run_id'].unique():
    maxTrialIndex = data_trial.loc[data_raw['run_id']==subject, ['trial_index']].max()
    if int(maxTrialIndex) < 15:
        subjects_failedSetups.append(subject)

data_trial.loc[data_trial['run_id'].isin(subjects_failedSetups), 
    [
        'run_id', 'prolificID'
    ]
].drop_duplicates()    

In [ ]:
data_trial = data_trial.loc[~data_trial['run_id'].isin(subjects_failedSetups), :]
print(data_trial['run_id'].unique())

### Duplicate Prolific ID's

In [ ]:
data_trial.loc[data_trial['run_id']==2002, 'prolificID'] = 'Tim2'

duplicates = data_trial.loc[:, ['prolificID', 'trial_index']].duplicated()
duplicateSubjects = data_trial.loc[duplicates, ['run_id', 'prolificID', 'trial_index']]

if len(duplicateSubjects) > 0:
    print('! Attention: Duplicate subjects: Check out the following: \n')
    print(duplicateSubjects)
else:
    print('Success: No duplicate subjects found')

### Numerics

In [ ]:
data_trial = convertToNumeric(data_trial, 
    [
        'run_id', 'subject', 'chinFirst', 'chin', 'task_nr', 'trial_index', # Int
        'key_press', 
        'x_pos', 'y_pos', 'time_elapsed', 'trial_duration',
        'rt',
        'window_width', 'window_height'
    ]
)

# data_et

## Compile

In [ ]:
def reformatYang(text):
    text = text.replace('$', ',')
    text = text.replace("relative-x", "x")
    text = text.replace("relative-y", 'y')
    text = text.replace('elapse-time', 't')
    text = (text[11:len(text)-1])
    return(text)

for subject in data_yang['run_id'].unique():
    for i in data_raw.loc[
        (data_raw['run_id']==subject) & 
        (pd.notna(data_raw['et_data'])) &
        ~(data_raw['et_data'].isin(['"', 'nan'])), 
        :].index:
        print('Reformat index: ' + str(i))
        data_raw.loc[i, 'et_data'] = reformatYang(data_raw.loc[i, 'et_data'])

In [ ]:
def textToDataframe(text):
    text = text.replace('$', ',')
    dataframe = pd.read_json(text, orient='records')
    return(dataframe)

def extractEyetrackingData(data):
    data_eyetracking = pd.DataFrame(columns=['x', 'y', 't'])
    data["et_data"] = data['et_data'].apply(str)

    for i in data.loc[
        (pd.notna(data['et_data'])) & 
        ~(data['et_data'].isin(['"', '[]', 'nan'])), 
        :].index:
        
        print('Start decoding Index ' + str(i))
        df = textToDataframe(data.loc[i, 'et_data'])        

        df["t_task"] = (df.loc[:, "t"] - df.loc[0, "t"])
        df['run_id'] = data.loc[i, 'run_id']
        df['trial_index'] = data.loc[i, 'trial_index']

        data_eyetracking = data_eyetracking.append(pd.DataFrame(data=df), ignore_index=True)
        print('Index ' + str(i) + ' extracted')
    return(data_eyetracking)

data_et = extractEyetrackingData(data_raw.loc[data_raw['run_id'].isin(data_trial['run_id'].unique()), :])

In [ ]:
data_et = convertToNumeric(data_et, ['x', 'y', 't', 't_task'])

In [ ]:
data_et.groupby(['run_id', 'trial_index'])['x', 'y'].count()

# data_subject

In [ ]:
df1 = data_raw.loc[
    :, 
    [
        'run_id',
        'age', 'browliner', 'browser', 'browser_version', 'degree', 'device',
        'ethnic', 'eyeliner', 'eyeshadow', 'gender', 'glasses', 'keptHead',
        'masquara', 'optionalNote', 'platform', 'platform_version', 'sight',
        'triedChin', 'user_agent', 'vertPosition', 'webcam_fps',
        'webcam_height', 'webcam_label', 'webcam_width'
    ]
].drop_duplicates()
    
df2 = data_trial.loc[
    : ,
    [
        'chinFirst', 'choiceTask_amountLeftFirst', 'chosenAmount',
        'chosenDelay', 'run_id', 'prolificID'
    ]
].drop_duplicates()
    
data_subject = df1.merge(df2, on='run_id', how='left') \
    .rename(columns={'age': 'birthyear'})

## Add prolific data

In [ ]:
data_prolific = pd.read_csv(r'C:/Users/User/Google Drive/Masterarbeit/Acquise/prolific_documents/prolific_export.csv') \
    .rename(columns={'participant_id': 'prolificID'})

data_subject = data_subject.merge(data_prolific, on='prolificID', how='left')

In [ ]:
data_subject['status'] = data_subject['status'].fillna('NOTPROLIFIC')

In [ ]:
data_subject.loc[:, ['run_id', 'prolificID']]

# Export data

In [ ]:
if not os.path.exists('./data_jupyter'):
    os.mkdir('./data_jupyter')

# data_et.to_csv("data_jupyter/data_et.csv", index=False, header=True)
data_trial.to_csv("data_jupyter/data_trial.csv", index=False, header=True)
# data_subject.to_csv("data_jupyter/data_subject.csv", index=False, header=True)

# Feedback

In [ ]:
print('Success! Script ran through')